In [3]:
import pandas as pd

rt_movie_details_df = pd.read_csv('zippedData/rt.movie_info.tsv.gz', sep='\t')

Changing Box Office values from object to float

In [75]:
rt_movie_details_df.dropna(subset=['box_office'], inplace=True)
rt_movie_details_df['box_office'] = rt_movie_details_df['box_office'].str.replace(',','')
rt_movie_details_df['box_office'] = rt_movie_details_df['box_office'].astype('float')

AttributeError: Can only use .str accessor with string values!

In [73]:
rt_movie_details_df.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio,main_genre,genre_list
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,600000.0,108 minutes,Entertainment One,Drama,"[Drama, Science Fiction and Fantasy]"
6,10,Some cast and crew from NBC's highly acclaimed...,PG-13,Comedy,Jake Kasdan,Mike White,"Jan 11, 2002","Jun 18, 2002",$,41032915.0,82 minutes,Paramount Pictures,Comedy,[Comedy]
7,13,"Stewart Kane, an Irishman living in the Austra...",R,Drama,Ray Lawrence,Raymond Carver|Beatrix Christian,"Apr 27, 2006","Oct 2, 2007",$,224114.0,123 minutes,Sony Pictures Classics,Drama,[Drama]
8,14,"""Love Ranch"" is a bittersweet love story that ...",R,Drama,Taylor Hackford,Mark Jacobson,"Jun 30, 2010","Nov 9, 2010",$,134904.0,117 minutes,NaN,Drama,[Drama]
15,22,Two-time Academy Award Winner Kevin Spacey giv...,R,Comedy|Drama|Mystery and Suspense,George Hickenlooper,Norman Snider,"Dec 17, 2010","Apr 5, 2011",$,1039869.0,108 minutes,ATO Pictures,Comedy,"[Comedy, Drama, Mystery and Suspense]"


In [74]:
rt_movie_details_df['genre_list'] = rt_movie_details_df['genre'].str.split('|')
rt_movie_details_df

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio,main_genre,genre_list
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,600000.0,108 minutes,Entertainment One,Drama,"[Drama, Science Fiction and Fantasy]"
6,10,Some cast and crew from NBC's highly acclaimed...,PG-13,Comedy,Jake Kasdan,Mike White,"Jan 11, 2002","Jun 18, 2002",$,41032915.0,82 minutes,Paramount Pictures,Comedy,[Comedy]
7,13,"Stewart Kane, an Irishman living in the Austra...",R,Drama,Ray Lawrence,Raymond Carver|Beatrix Christian,"Apr 27, 2006","Oct 2, 2007",$,224114.0,123 minutes,Sony Pictures Classics,Drama,[Drama]
8,14,"""Love Ranch"" is a bittersweet love story that ...",R,Drama,Taylor Hackford,Mark Jacobson,"Jun 30, 2010","Nov 9, 2010",$,134904.0,117 minutes,NaN,Drama,[Drama]
15,22,Two-time Academy Award Winner Kevin Spacey giv...,R,Comedy|Drama|Mystery and Suspense,George Hickenlooper,Norman Snider,"Dec 17, 2010","Apr 5, 2011",$,1039869.0,108 minutes,ATO Pictures,Comedy,"[Comedy, Drama, Mystery and Suspense]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1541,1980,A band of renegades on the run in outer space ...,PG-13,Action and Adventure|Science Fiction and Fantasy,Joss Whedon,Joss Whedon,"Sep 30, 2005","Dec 20, 2005",$,25335935.0,119 minutes,Universal Pictures,Action and Adventure,"[Action and Adventure, Science Fiction and Fan..."
1542,1981,"Money, Fame and the Knowledge of English. In I...",NR,Comedy|Drama,Gauri Shinde,Gauri Shinde,"Oct 5, 2012","Nov 20, 2012",$,1416189.0,129 minutes,Eros Entertainment,Comedy,"[Comedy, Drama]"
1545,1985,A woman who joins the undead against her will ...,R,Horror|Mystery and Suspense,Sebastian Gutierrez,Sebastian Gutierrez,"Jun 1, 2007","Oct 9, 2007",$,59371.0,98 minutes,IDP Distribution,Horror,"[Horror, Mystery and Suspense]"
1546,1986,Aki Kaurismaki's The Man Without a Past opens ...,PG,Art House and International|Comedy|Drama,NaN,NaN,"Aug 30, 2002","Oct 7, 2003",$,794306.0,97 minutes,NaN,Art House and International,"[Art House and International, Comedy, Drama]"


Test run for getting all animation directors

In [79]:
animation_directors_df = rt_movie_details_df[rt_movie_details_df.apply(lambda x: 'Animation' in x['genre_list'], axis=1)]


In [80]:
animation_directors_df

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio,main_genre,genre_list
271,357,Planet 51 is a galactic-sized animated alien a...,PG,Action and Adventure|Animation|Comedy|Kids and...,Jorge Blanco,Joe Stillman,"Nov 20, 2009","Mar 9, 2010",$,42194060.0,91 minutes,Sony Pictures/TriStar Pictures,Action and Adventure,"[Action and Adventure, Animation, Comedy, Kids..."
346,452,"The stars have aligned in an ominous sign, and...",PG,Action and Adventure|Animation|Science Fiction...,Kevin Munroe,Kevin Munroe,"Mar 23, 2007","Aug 7, 2007",$,54132596.0,86 minutes,Warner Bros. Pictures,Action and Adventure,"[Action and Adventure, Animation, Science Fict..."
370,482,"Take out the trash, eat your broccoli-who need...",PG,Action and Adventure|Animation|Comedy|Science ...,Simon Wells,Simon Wells|Wendy Wells,"Mar 11, 2011","Aug 9, 2011",$,21379315.0,88 minutes,Walt Disney Pictures,Action and Adventure,"[Action and Adventure, Animation, Comedy, Scie..."
375,488,"A recent college graduate, Barry B. Benson wan...",PG,Animation|Comedy|Kids and Family,NaN,Spike Feresten|Jerry Seinfeld|Barry Marder|And...,"Nov 2, 2007","Mar 11, 2008",$,126597121.0,90 minutes,Paramount Pictures,Animation,"[Animation, Comedy, Kids and Family]"
431,564,"For super-dog ""Bolt"", every day is filled with...",PG,Action and Adventure|Animation|Comedy,NaN,NaN,"Nov 21, 2008","Mar 24, 2009",$,114053579.0,96 minutes,Walt Disney Pictures,Action and Adventure,"[Action and Adventure, Animation, Comedy]"
509,667,"In the 3D animated comedy, The Angry Birds Mov...",PG,Animation|Kids and Family,Clay Kaytis|Fergal Reilly,Jon Vitti,"May 20, 2016","Aug 16, 2016",$,107506776.0,95 minutes,Sony Pictures,Animation,"[Animation, Kids and Family]"
633,816,Two mismatched wolves embark on a cross-countr...,PG,Action and Adventure|Animation|Comedy|Kids and...,Anthony Bell|Ben Gluck|Benjamin Gluck,Steve Moore|Chris Denk,"Sep 17, 2010","Jan 11, 2011",$,25077977.0,88 minutes,Lionsgate Films,Action and Adventure,"[Action and Adventure, Animation, Comedy, Kids..."
913,1180,Despite having recently presided over a very s...,PG,Animation|Kids and Family|Science Fiction and ...,Henry Selick|Tim Burton,Tim Burton|Caroline Thompson|Michael McDowell,"Oct 24, 1993","Dec 2, 1997",$,8616662.0,76 minutes,Touchstone Pictures,Animation,"[Animation, Kids and Family, Science Fiction a..."
958,1239,Something mysterious is happening in the Hundr...,G,Animation|Kids and Family,Frank Nissen,Brian Hohlfeld|Evan Spiliotopoulos,"Feb 11, 2005","May 24, 2005",$,18013938.0,67 minutes,Buena Vista,Animation,"[Animation, Kids and Family]"
1007,1305,"A cow learns to walk like a man, both literall...",PG,Animation|Comedy|Kids and Family,Steve Oedekerk,Steve Oedekerk,"Aug 4, 2006","Dec 12, 2006",$,72601713.0,90 minutes,Paramount Pictures,Animation,"[Animation, Comedy, Kids and Family]"


In [81]:
animation_directors_df = animation_directors_df.groupby('director').mean().sort_values(by='box_office', 
                                                                                       ascending=False)

In [82]:
animation_directors_df

,id,box_office
director,,
Gore Verbinski,1601,123188232.0
Clay Kaytis|Fergal Reilly,667,107506776.0
Peter Lord|Nick Park,1418,106604314.0
Conrad Vernon|Greg Tiernan,1890,97661826.0
Steve Oedekerk,1305,72601713.0
Walt Becker,1383,66580191.0
Kevin Munroe,452,54132596.0
Jorge Blanco,357,42194060.0
Ron Clements|John Musker,1690,38120554.0


In [83]:
def director_genre_dataframe(genre):
    df = rt_movie_details_df[rt_movie_details_df.apply(lambda x: genre in x['genre_list'], axis=1)]
    genre_df = df.groupby('director').mean().sort_values(by='box_office', ascending=False)
    return genre_df

In [89]:
director_genre_dataframe('Action and Adventure')

,id,box_office
director,,
Peter Jackson,1136,303001229.0
Sam Mendes,1525,299300000.0
Peter Berg,1877,227946274.0
Bryan Singer,1600,214813155.0
Justin Lin,1722,209805005.0
...,...,...
Ciro Guerra,1976,1320005.0
Rohit Shetty,490,1231550.0
Kyle Newman,833,700000.0


In [90]:
def writer_genre_dataframe(genre):
    df = rt_movie_details_df[rt_movie_details_df.apply(lambda x: genre in x['genre_list'], axis=1)]
    genre_df = df.groupby('writer').mean().sort_values(by='box_office', ascending=False)
    return genre_df

In [96]:
writer_genre_dataframe('Action and Adventure')

Index(['Peter Jackson|Philippa Boyens|Guillermo del Toro|Fran Walsh',
       'John Logan|Robert Wade|Neal Purvis',
       'Gene Roddenberry|Alex Kurtzman|Roberto Orci',
       'Josh Friedman|David Koepp',
       'Vincent Gilligan|Vince Gilligan|Vy Vincent Ngo'],
      dtype='object', name='writer')